**1. Corporation wants to investigate the best-selling item across stores**
**2. The Best selling item in All Stores**
**3. Best selling bundled items in stores**
**4. Best performing stores via sales**
**5. The best selling items in the best selling stores**
**6. Worse performing products and its corresponding stores**
**7. Worse performing stores by sales**

In [4]:
import pandas as pd 
import random
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [6]:
df = pd.read_csv('sales_data.csv') #load csv file
baskets_per_store = df.groupby(['StoreID', 'OrderID'])['Product Name'].apply(list)

In [ ]:
def get_best_selling_item_per_store(df, store_id):
    store_data = df[df['StoreID'] == store_id]
    best_selling_item = store_data['Product Name'].value_counts().head(1)
    return best_selling_item

random_stores = random.sample(df['StoreID'].unique().tolist(), 10)

best_selling_items_per_store = []

for store_id in random_stores:
    best_selling_item = get_best_selling_item_per_store(df, store_id)
    best_selling_items_per_store.append((store_id, best_selling_item.index[0], best_selling_item.values[0]))

best_selling_df = pd.DataFrame(best_selling_items_per_store, columns=['StoreID', 'BestSellingItem', 'SalesCount'])

top_10_stores = best_selling_df.sort_values(by='SalesCount', ascending=False).head(10)

for idx, row in top_10_stores.iterrows():
    print(f"Store: {row['StoreID']}    Best selling item: {row['BestSellingItem']} ..  Sold {row['SalesCount']} times")

In [ ]:
def get_best_selling_items_across_organization(df):
    return df['Product Name'].value_counts().head(10)

top_selling_items = get_best_selling_items_across_organization(df)

header = "Best Selling Items Across All Stores:"
separator = "=" * 37
print(f"{header}\n{separator}")

for idx, (item, count) in enumerate(top_selling_items.items(), start=1):
    print(f"{idx}. {item} - Sold {count} times")

In [ ]:
random_stores = random.sample(baskets_per_store.index.get_level_values(0).unique().tolist(), 10)

report = []

for store_id in random_stores:
    transactions = baskets_per_store[store_id].tolist()

    te = TransactionEncoder()
    store_df = pd.DataFrame(te.fit_transform(transactions), columns=te.columns_)

    frequent_itemsets = apriori(store_df, min_support=0.05, use_colnames=True)

    bundled_itemsets = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x) > 1)]
    top_itemsets = bundled_itemsets.nlargest(5, 'support')

    report.append(f"Top 5 Bundled Items for Store {store_id}:")
    report.extend(
        f"{', '.join(row['itemsets'])} - Purchased together in {row['support']*100:.2f}% of orders."
        for _, row in top_itemsets.iterrows()
    )
    report.append("".center(50, "-"))

print("\n".join(report))


In [ ]:
df['Product Price'] = df['Product Price'].replace({'$': '', ',': ''}, regex=True).astype(float)

total_sales_per_order = df.groupby(['StoreID', 'OrderID'])['Product Price'].sum().reset_index()

total_sales_per_store = total_sales_per_order.groupby('StoreID')['Product Price'].sum().reset_index()

top_10_stores_by_sales = total_sales_per_store.sort_values(by='Product Price', ascending=False).head(10)

top_10_stores_by_sales['Top'] = range(1, len(top_10_stores_by_sales) + 1)

for idx, row in top_10_stores_by_sales.iterrows():
    print(f"Top {int(row['Top'])}: Store {int(row['StoreID'])}   Total Sales: ${row['Product Price']:.2f}")

In [ ]:
best_selling_items_per_store = df.groupby(['StoreID', 'Product Name']).size().reset_index(name='Count')

best_selling_items_per_store = best_selling_items_per_store.sort_values(by=['StoreID', 'Count'], ascending=[True, False])

top_best_selling_items = best_selling_items_per_store.drop_duplicates('StoreID', keep='first')

top_10_stores = top_best_selling_items.sort_values(by='Count', ascending=False).head(10)

top_10_stores['Rank'] = range(1, len(top_10_stores) + 1)

for idx, row in top_10_stores.iterrows():
    print(f"Top {row['Rank']}: Store {row['StoreID']}   Best-Selling Item: {row['Product Name']} with {row['Count']} purchases")
     

In [ ]:
product_report = []
product_report.append("\nBottom 3 Performing Products by Sales:")
product_report.append("-" * 50)

worst_products = df['Product Name'].value_counts().tail(3)
for product, sales in worst_products.items():
    stores_with_product = df[df['Product Name'] == product]['StoreID'].unique()[:5]  
    stores_list = ", ".join(map(str, stores_with_product))
    product_report.append(f"{product} - Sold {sales} times in stores: {stores_list}")

store_report = []
store_report.append("\nTop 10 Worst-Performing Stores by Total Sales:")
store_report.append("-" * 50)

total_sales_per_store = df.groupby('StoreID').size()
worse_stores = total_sales_per_store.nsmallest(10)

for store, sales in worse_stores.items():
    store_report.append(f"Store {store} - Total Sales: {sales}")

print("\n".join(product_report))


In [ ]:
product_report = []
product_report.append("\nBottom 3 Performing Products by Sales:")
product_report.append("=" * 50)

worst_products = df['Product Name'].value_counts().tail(3)
for product, sales in worst_products.items():
    stores_with_product = df[df['Product Name'] == product]['StoreID'].unique()[:5]  
    stores_list = ", ".join(map(str, stores_with_product))
    product_report.append(f"{product} - Sold {sales} times in stores: {stores_list}")
    product_report.append("-" * 50)

store_report = []
store_report.append("\nTop 10 Worst-Performing Stores by Total Sales:")
store_report.append("-" * 50)

total_sales_per_store = df.groupby('StoreID').size()
worse_stores = total_sales_per_store.nsmallest(10)

for store, sales in worse_stores.items():
    store_report.append(f"Store {store} - Total Sales: {sales}")


print("\n".join(store_report))
